# 1) Weather info.

This notebooks contains the code for the geographical analysis of the area of interest. We'll gather information about:



1.   The wind of the targeting area, such as speed and direction.
2.   The temperature.
3.   The possibility of rain.
4.   The humidity.


These factors are important for the prediction of a wildfire.

In [ ]:
# Import libraries we need to use

import requests
import lxml
from bs4 import BeautifulSoup
import re
import math
from scipy.spatial import distance
import numpy as np

In [ ]:
# Create and access url for: Windfinder

# 38.3848
# 23.8003
# url_wind = "https://www.windfinder.com/forecast/#38.3848/23.8003"
url_wind = "https://www.windfinder.com/forecast/eretria_central_greece_greece"

# url = "https://www.windfinder.com/forecast/nigel_s_place"
# url = "https://api-windfinder-pro.p.rapidapi.com/spots/%7B520805b666mshcb0dfbf36037028p1a5d27jsn41d130eacb2e%7D/forecasts"
# https://api-windfinder-pro.p.rapidapi.com/spots/%7Bspot_id%7D/forecasts
# 520805b666mshcb0dfbf36037028p1a5d27jsn41d130eacb2e
f = requests.get(url_wind)

In [ ]:
# Parse webpage (Windfinder)

# print(f.content)
# print(f.text)
#Get the JSON info
soup = BeautifulSoup(f.content,'lxml')

#Convert it to a String
soup_string = str(soup)

#Weather info
weather_info = soup_string.split("cc:",1)[1]

#Split word
spl_word = '});'

#Final infos we care about
res = weather_info.split(spl_word)[0]

#Printing the result
print(res)

#Insert to variables  1) wind's direction
#                     2) wind's speed
#                     3) temperature 

wind_direction  = re.search('wd: (.*),', res)
wind_speed      = re.search('ws: (.*),', res)
temperature     = re.search('at: (.*),', res)

wind_direction  = str(wind_direction.group(1))
wind_speed      = str(wind_speed.group(1))
temperature     = str(temperature.group(1))

print(wind_direction, wind_speed, temperature)


 {
        lastUpdateTimeLocal: "2022-09-28T12:00:00+03:00",
       
        isNight: false, 

        isReport: false,

        wd: 240,
        ws: 5,

        cl: 0,
        at: 28.74,
        ap: 977.15,
        p: null,
        pt: null    },

240 5 28.74


In [ ]:
# Create and access url for: weather.com
# Fetching Humidity and Cloud prediction

url_weather = "https://weather.com/weather/today/l/37.98,23.74?par=google"

f = requests.get(url_weather)

In [ ]:
#Get the JSON info
soup = BeautifulSoup(f.content,'lxml')

#Convert it to a String
soup_string = str(soup)

#Extract the humidity percentage of the JSON
string = "relativeHumidity"

index = soup_string.find(string)

hum_string = soup_string[index:index+22]

####
humidity = re.sub("[^0-9]", "", hum_string)
####
print(humidity)

str_cloud = '"data-testid="wxPhrase"'
cloud_report_re = re.search('data-testid="wxPhrase">(.*)</div>', soup_string)
####
cloud_report = (cloud_report_re.group(1)).partition('<')[0]
####
print(cloud_report)

45
Fair


So, at this point, we have all the necessary information we need about the weather. Below, we are printing these.

In [ ]:
print("Wind speed is: ", wind_speed)
print("Wind direction is: ", wind_direction)
print("Temperature is : ", temperature, " °C")
print("Humidity is: ", humidity, "\b%")
print("Cloud report is: ", cloud_report)

Wind speed is:  5
Wind direction is:  238
Temperature is :  28.74  °C
Humidity is:  45 %
Cloud report is:  Fair


In [ ]:
# Previous was: 9 and real is 358, so: -11
# Previous was: 99 and real is 85, so: -14
# wind_dir_real = int(wind_direction) - 11
# print(wind_dir_real)
# if(wind_dir_real < 0 ):
#   wind_direction = 360 - wind_dir_real 
# else :
#   wind_direction = wind_dir_real
# print(wind_direction)
# wind_direction = "358"
# wind_direction = int("90")

In [ ]:
# wind_direction = 90 - int(wind_direction)
# wind_direction = 50

# 2) Polygon creation command (GeoGebra).

In order to find the optimal positions of the drone in the polygon area, we need to do some work on GeoGebra.
To be more specific, we need to create the polygon (That's what are going to do here), and after find the intersection points.

In [ ]:
mystr = input('please enter string: ')
print(mystr)

please enter string: 0: [23.800966156784177,38.385898843453276] 1: [23.800946040216566,38.385898843453276] 2: [23.800919218126417,38.385891484914204] 3: [23.800915194812895,38.38586415319112] 4: [23.80090178376782,38.38583156535393] 5: [23.80088166720021,38.385804233608205] 6: [23.800857527319074,38.38576218474846] 7: [23.800830705228925,38.38570331630373] 8: [23.800794072814146,38.385671801017004] 9: [23.800741769738355,38.38562975208027] 10: [23.800710924334684,38.38560242025826] 11: [23.800666667885938,38.38556247373063] 12: [23.800626434750715,38.38554670535838] 13: [23.80059827155606,38.38556772985395] 14: [23.800545968480268,38.38554144923351] 15: [23.800496347613493,38.38553198820785] 16: [23.800431974597135,38.38551516860355] 17: [23.80038235373036,38.38550465634889] 18: [23.80030322856442,38.385474170801736] 19: [23.800237514443555,38.38543527542982] 20: [23.800205327935377,38.38539217512839] 21: [23.80026066210846,38.385373273045275] 22: [23.800259321003953,38.385341736214826

In [ ]:
my_list = mystr.split(" ")
print(my_list)

['0:', '[23.800966156784177,38.385898843453276]', '1:', '[23.800946040216566,38.385898843453276]', '2:', '[23.800919218126417,38.385891484914204]', '3:', '[23.800915194812895,38.38586415319112]', '4:', '[23.80090178376782,38.38583156535393]', '5:', '[23.80088166720021,38.385804233608205]', '6:', '[23.800857527319074,38.38576218474846]', '7:', '[23.800830705228925,38.38570331630373]', '8:', '[23.800794072814146,38.385671801017004]', '9:', '[23.800741769738355,38.38562975208027]', '10:', '[23.800710924334684,38.38560242025826]', '11:', '[23.800666667885938,38.38556247373063]', '12:', '[23.800626434750715,38.38554670535838]', '13:', '[23.80059827155606,38.38556772985395]', '14:', '[23.800545968480268,38.38554144923351]', '15:', '[23.800496347613493,38.38553198820785]', '16:', '[23.800431974597135,38.38551516860355]', '17:', '[23.80038235373036,38.38550465634889]', '18:', '[23.80030322856442,38.385474170801736]', '19:', '[23.800237514443555,38.38543527542982]', '20:', '[23.800205327935377,

In [ ]:
polygon = "Polygon("
i = 1
while i < len(my_list):
  print(my_list[i])
  each_cord = "("
  j = 0
  coords = my_list[i].split(",")
  while j < len(coords):
    # print(coords[j])
    # print(my_list[j])
    fixed_cord = re.sub('[^a-zA-Z0-9 \n\.]', '', coords[j])
    if (j == 1):
      each_cord += ","
    each_cord+= fixed_cord
    j +=1
  each_cord += ")"
  polygon += each_cord + ","
  i += 2

polygon += ")"

size = len(polygon)
true_poly = polygon.replace(polygon[size -2:], ")")

print(true_poly)

[23.800966156784177,38.385898843453276]
[23.800946040216566,38.385898843453276]
[23.800919218126417,38.385891484914204]
[23.800915194812895,38.38586415319112]
[23.80090178376782,38.38583156535393]
[23.80088166720021,38.385804233608205]
[23.800857527319074,38.38576218474846]
[23.800830705228925,38.38570331630373]
[23.800794072814146,38.385671801017004]
[23.800741769738355,38.38562975208027]
[23.800710924334684,38.38560242025826]
[23.800666667885938,38.38556247373063]
[23.800626434750715,38.38554670535838]
[23.80059827155606,38.38556772985395]
[23.800545968480268,38.38554144923351]
[23.800496347613493,38.38553198820785]
[23.800431974597135,38.38551516860355]
[23.80038235373036,38.38550465634889]
[23.80030322856442,38.385474170801736]
[23.800237514443555,38.38543527542982]
[23.800205327935377,38.38539217512839]
[23.80026066210846,38.385373273045275]
[23.800259321003953,38.385341736214826]
[23.800220428973237,38.385312301827355]
[23.80017617252449,38.38529863585767]
[23.800146668225327,38.

# 3) Line equation based on wind angle.

In order to compute the optimal position for the drone, we must draw `n` lines in the direction of the wind.

Wind direction (express in deegres), will give us the wind direction. Based on this, we'll compute the line equation, which will be something like:

$y = a*x + b$.

But, because we need the angle of the line with y-axis, the equation will be like: $a * y = x + b$

Since, we don't care now for the line position in x-axis, b will be represent a series of b's => parallel lines.

So: $\mathbf{b}$ = ${b1, b2, ... , bn}$. 

b now, will represent a vector of numbers. 

In [ ]:
wind_direction = int(wind_direction)
a = math.tan( math.radians(wind_direction))

line_eq = str(a) + 'y = x' 

print(line_eq)

1.19175359259421y = x


# 4) Creating b vector.

The *b* vector, it's one of the most important parts here. 

What we need to do here, it's to create n lines. In order to do that, we need a b vector, that represent parallel lines of the starting line, n/2 left and n/2 right. (We have an edge case here, where angle is 90°, we'll talk about this later).

The centre of the polygon, after computing it based on graph, is: `23.8022`.

In [ ]:
#Intersection point between line_eq and centre of the polygon.
centre_x = 23.8005
centre_y = 38.3849

intersection_x = centre_y * a
intersection_y = centre_y

#Computing the euclidean distance between these 2 points now.

point_a = (centre_x, centre_y)
point_b = (intersection_x, intersection_y)

dist = distance.euclidean(point_a, point_b)
print(dist)

21.944842476369494


In [ ]:
#So the line that comes through the centre of the polygon is:
# print(wind_direction)
if(int(wind_direction) > 180):
  sign = " - "
else:
  sign = " + "

# sign= " + "
# line_centre = line_eq + ' - ' + str(dist)
line_centre = line_eq + sign + str(dist)
print(line_centre)

1.19175359259421y = x + 21.944842476369494


In [ ]:
#Let's create the lines base on b vector:
#We changing the 4th decimal point.

#Lines in the east side of polygon
right_lines = []

#Lines in the west side of polygon
left_lines  = []

#The number of lines we want to produce
range_lines = 20
half_step = int(range_lines/2)

#b vector
line_cntr = str(dist)[0:7]
# print(line_cntr)

line_cntr_int = float(line_cntr)

for x in range(2):
  if(x == 0):
    for dec in range(half_step):
      # print(dec)
      dec_float = float(dec) / 10000
      b_i = line_cntr_int + dec_float
      str_bi = str(b_i)
      b_i = str_bi[0:7]
      right_lines.append(float(b_i))
  if(x == 1):
    for dec in range(half_step):
      dec_float = float(dec) / 10000
      b_i = line_cntr_int - dec_float
      str_bi = str(b_i)
      b_i = str_bi[0:7]
      left_lines.append(float(b_i))    

for line_ in range(len(right_lines)):
  new_line = line_eq + sign + str(right_lines[line_])
  right_lines[line_] = new_line

for line_ in range(len(left_lines)):
  new_line = line_eq + sign + str(left_lines[line_])
  left_lines[line_] = new_line

print("Right lines are: ", right_lines, "\n")
print("Left lines are: ", left_lines, "\n")
# print(left_lines)

Right lines are:  ['1.19175359259421y = x + 21.9448', '1.19175359259421y = x + 21.9449', '1.19175359259421y = x + 21.945', '1.19175359259421y = x + 21.9451', '1.19175359259421y = x + 21.9452', '1.19175359259421y = x + 21.9453', '1.19175359259421y = x + 21.9454', '1.19175359259421y = x + 21.9455', '1.19175359259421y = x + 21.9456', '1.19175359259421y = x + 21.9457'] 

Left lines are:  ['1.19175359259421y = x + 21.9448', '1.19175359259421y = x + 21.9447', '1.19175359259421y = x + 21.9446', '1.19175359259421y = x + 21.9445', '1.19175359259421y = x + 21.9444', '1.19175359259421y = x + 21.9443', '1.19175359259421y = x + 21.9442', '1.19175359259421y = x + 21.9441', '1.19175359259421y = x + 21.944', '1.19175359259421y = x + 21.9439'] 



# 5) Calculating the maximum area.

In order to find the starting point(s) of the drone placement, we need to calculate the area that it's been created by the intersection points, within the lines and the Polygon. In order to do that, we're going to compute the Euclidean distance between the interesction points that are being created.

In [ ]:
# A function to create the formal way of points (from string to float)
def createPoint(point):

  point = point.split("=",1)[1]  
  print(point, "\n")
  realPoint = []
  sPoint = point.split(",")
  for i in range(2):
    point_i = sPoint[i].replace("(", "")
    point_i = point_i.replace(")", "")
    point_i = float(point_i)
    realPoint.append(point_i)
  return realPoint

We start within the 'most' right line, to the 'most' left line.

In [ ]:
#Within this variable we're going to save the computed areas between the interesection points and the polygon
areaArr = []

for line in range(range_lines - 1):
  point = ""
  pointsNum = 0
  tempArea = 0
  point = input("Enter your point: ")
  if(point=="stop"):
    break
  while(point!= "next"):
    pointsNum+=1
    if(pointsNum==1):
      point_a = createPoint(point)
    else:
      point_b = createPoint(point)
      tempArea += distance.euclidean(point_a, point_b)
      pointsNum = 0
    point = input("Enter your point: ")

  areaArr.append(tempArea)

print(areaArr)


Enter your point: Ι_{3}=(23.799982467,38.385185286)
(23.799982467,38.385185286) 

Enter your point: Κ_{3}=(23.8001378326,38.3853156532)
(23.8001378326,38.3853156532) 

Enter your point: Λ_{3}=(23.8002221715,38.385386422)
(23.8002221715,38.385386422) 

Enter your point: Μ_{3}=(23.8003467118,38.3854909237)
(23.8003467118,38.3854909237) 

Enter your point: next
Enter your point: Γ_{3}=(23.7998721859,38.3850088392)
(23.7998721859,38.3850088392) 

Enter your point: Δ_{3}=(23.8004953357,38.385531724)
(23.8004953357,38.385531724) 

Enter your point: next
Enter your point: Ψ_{2}=(23.7996523953,38.3847405031)
(23.7996523953,38.3847405031) 

Enter your point: Ω_{2}=(23.8010550747,38.3859174908)
(23.8010550747,38.3859174908) 

Enter your point: next
Enter your point: Τ_{2}=(23.79965841,38.38466164)
(23.79965841,38.38466164) 

Enter your point: Υ_{2}=(23.8011346897,38.3859003857)
(23.8011346897,38.3859003857) 

Enter your point: next
Enter your point: Ο_{2}=(23.7995778592,38.3845101399)
(23.799577

In [ ]:
# areaArr = [0.0008503216989254822, 0.0010040706043823006, 0.0010020311238971371, 0.0010184337916585326, 0.0010717509787716284, 0.001056332691280226, 0.0011878141069283656, 0.0011979517344432701, 0.0012411878159957175, 0.001275196459303373, 0.00123394727909091, 0.001171035518645748, 0.0011191407822180681, 0.001072771523395908, 0.0010767619372013268, 0.0011167813142056594, 0.001182113359183252, 0.0011459100065514052, 0.0010592133261730608]
print(areaArr)

[0.00036539117810925175, 0.0008134643122572012, 0.0018310678701589918, 0.0019271462481872353, 0.002143119614203304, 0.0023594407051614804, 0.0025725451706732853, 0.0027427164206417247, 0.0027851904674536304, 0.002725431393553368, 0.0028386697191622595, 0.002769324135484416, 0.002801008465085499, 0.002762218316756403, 0.0028211391795513064, 0.0027599270579579415, 0.0015156049992716895, 0.0008692247584233422, 6.020788420344145e-05]


In [ ]:
def sortDistArr(arr):
  arr = np.argsort(arr)
  sortArr = arr[::-1]
  return sortArr

In [ ]:
indexArray = sortDistArr(areaArr)
print(indexArray)

[10 14 12  8 11 13 15  7  9  6  5  4  3  2 16 17  1  0 18]


# 6) Finding the nB(number of Bateries) best lines.

So, we calculated the distances between the interception points and the polygon.
Now, we must find which lines are those that cover the maximum area of the polygon.

In [ ]:
# Within this function, we're finding the best lines in the polygon.
def bestLines(arrIndex, nB):
  lines = []
  bestLines = []

  right_lines_ = right_lines[::-1]
  lines.append(right_lines_)

  # We remove the central line, because it's a duplicate
  left_lines.remove(left_lines[0])

  print(left_lines)
  lines.append(left_lines)

  nbLines = arrIndex[0:nB]

  # We make the 2D list -> 1D. *Flatten
  lines = [j for sub in lines for j in sub]

  print("Lines are: ", lines)
  print(nbLines)

  bestLines = [] 
  for index in nbLines:
    print(index)
    bestLines.append(lines[index])

  return bestLines

In [ ]:
# Number of bateries are also the number of tries we have to fly our drone.
nB = 3

best_Lines = bestLines(indexArray, nB)
print(best_Lines)

1.19175359259421y = x + 21.9447
['1.19175359259421y = x + 21.9446', '1.19175359259421y = x + 21.9445', '1.19175359259421y = x + 21.9444', '1.19175359259421y = x + 21.9443', '1.19175359259421y = x + 21.9442', '1.19175359259421y = x + 21.9441', '1.19175359259421y = x + 21.944', '1.19175359259421y = x + 21.9439']
Lines are:  ['1.19175359259421y = x + 21.9457', '1.19175359259421y = x + 21.9456', '1.19175359259421y = x + 21.9455', '1.19175359259421y = x + 21.9454', '1.19175359259421y = x + 21.9453', '1.19175359259421y = x + 21.9452', '1.19175359259421y = x + 21.9451', '1.19175359259421y = x + 21.945', '1.19175359259421y = x + 21.9449', '1.19175359259421y = x + 21.9448', '1.19175359259421y = x + 21.9446', '1.19175359259421y = x + 21.9445', '1.19175359259421y = x + 21.9444', '1.19175359259421y = x + 21.9443', '1.19175359259421y = x + 21.9442', '1.19175359259421y = x + 21.9441', '1.19175359259421y = x + 21.944', '1.19175359259421y = x + 21.9439']
[10 14 12]
10
14
12
['1.19175359259421y = x + 2

# 7) Design of the area (Circle) that our drone will make it's flight.

After our computations, we've found that: 

1m = 0.00000954495 mm, in GeoGebra. So, in order to solve our equations, we'll take that as a fact and we'll proceed to compute the equation of the Circle that we want.

The Equation of the Circle is: $(x-x_0)^2 + (y - y_0)^2 = r^2$.



In [ ]:
# A function to create the formal way of points (from string to float)
def createPointForCircle(point):

  # point = point.split("=",1)[1]  
  # print(point, "\n")
  realPoint = []
  sPoint = point.split(",")
  for i in range(2):
    point_i = sPoint[i].replace("(", "")
    point_i = point_i.replace(")", "")
    point_i = float(point_i)
    realPoint.append(point_i)
  return realPoint

In [ ]:
# Let's find the equation of the circle.
# First, we'll find the centre that we want, making the first circle.
# That's why we'll have 2 iterations

Metric = 0.00000954495
RangeOfDrone = input("Enter the range of drone: ")
Circles = []

for i in range(2):

  # RangeOfDrone = 50
  #It's not / 2, but / 1.
  Radius = int(RangeOfDrone) / 1

  # Metric = 0.00000954495

  point = input("Enter the point(centre) of the circle: ")
  point = createPointForCircle(point)
  r = Metric * Radius
  # point = (23.8013364451, 38.3853983989)

  print(r)
  CircleEq = "(x - " + str(point[0]) + ") ^ 2 + (y - " + str(point[1]) + ") ^ 2 = " + str(float(pow(r , 2)))
  print(CircleEq)
  Circles.append(CircleEq)

Enter the range of drone: 100
Enter the point(centre) of the circle: (23.8013364451, 38.3853983989)
0.0009544950000000001
(x - 23.8013364451) ^ 2 + (y - 38.3853983989) ^ 2 = 9.110607050250002e-07
Enter the point(centre) of the circle: (23.8009708524, 38.3850916303)
0.0009544950000000001
(x - 23.8009708524) ^ 2 + (y - 38.3850916303) ^ 2 = 9.110607050250002e-07


In [ ]:
for C in Circles:
  print(C,"\n")

(x - 23.8013364451) ^ 2 + (y - 38.3853983989) ^ 2 = 2.2776517625625005e-07 

(x - 23.8009708524) ^ 2 + (y - 38.3850916303) ^ 2 = 2.2776517625625005e-07 



(x - 23.8009708524) ^ 2 + (y - 38.3850916303) ^ 2 = 0.0000009110607050250002


# 8) Flight Path: The flight that our drone will do.

So, our drone will be driven by the user's pc and navigate through his control. Nevertheless, we'll also create a cost function that will evaluate in real-time if our decisions are right or not. 
We also must considerate our limits of the polygon. Our dorne must fly within the limits and to not fly far away from there.

## $C(x,y) = w * d(x,y) + t(x,y).$

### 1) Weight parameter: $w = \mathbb{R} - \{0\}.$


This parameter will determine how "important" is the $d(x,y)$ factor. *We'll test many numbers after our expirements, maybe w factor will be not a fixed value.

### 2) Distance parameter: $d( p(x,y)  ,  set(inc_p)).$

Where $p(x,y)$ is the current position of our drone and $set(inc_p)$ is the 'set' of the intersection points of the lines that are in the same direction of the wind, and they intersect our polygon.

### 3) Time parameter: $t( d(x,y) > dis('danger') ).$

In particular, the time parameter counts the seconds, when the value of : $d(x,y)$ is bigger than the 'distance of danger', which it means, the maximum distance where the drone is too far away from the intersection points. 
This parameter help us 'contain' our drone close by the dangerous areas, with an option of freedom to explore different areas of interests near by. 
This parameter may change, because of the speed of wind. If the speed of wind is 'high' enough, when high will be meausured by kts (which is the measurement unit of the speed of wind), the distance of range will be smaller. 